### **Problem Statement:** 
We aim to solve **1D steady-state flow problems** in homogeneous porous media using the **Physics-Informed Neural Networks (PINN)** approach for solving PDEs, implemented with the JAX library. 



### **Domain**:
$$ 
\Omega = \{ x \mid 0 \leq x \leq L \}
$$



### **Governing Equation**:
The 1D steady-state flow in homogeneous porous media is governed by Darcy's law as follows:
<!-- $$ 
\frac{\partial}{\partial x} \left( K \frac{\partial h}{\partial x} \right) = 0, \quad \text{in } \Omega.
$$ 

For a homogeneous medium where hydraulic conductivity, K is constant, this simplifies to: -->
$$ 
\frac{\partial^2 h}{\partial x^2} = 0, \quad \text{in } \Omega.
$$ 



### **Boundary Conditions**

**Dirichlet Boundary Conditions** (Prescribed pressure on the left and right boundaries):  
$$ 
h(0) = h_1, \quad h(L) = h_2.
$$



### **Objective**:
We solve this problem using Physics-Informed Neural Networks (PINN) to predict the head, h(x),  across the domain and compare it with the analytical solution for the same configuration.


---
**Developed by**:
Lal Mamud, Postdoc - Subsurface Modeler, Environmental Subsurface Science Group, Energy & Environment Division, Pacific Northwest National Laboratory, Richland, WA, USA.

**Mentors**:
Maruti K. Mudunuru and Satish Karra

---


## 1. Import required Python libraries 

In [12]:
import os
import sys
import numpy as np
import jax
import jax.numpy as jnp
from jax import random, jit, grad, vmap, jacfwd, jacrev
import optax
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

## 2. Define Domain and Boundary Conditions, Generate Collocation Points

In [25]:
xmin = 0.0  # Left boundary of the domain
xmax = 1.0  # Right boundary of the domain

lb_head = 1  # Head at the left boundary
rb_head = 0.9  # Head at the right boundary
bcs = [lb_head, rb_head]  # Boundary conditions as a list

n_colloc = 100  # Number of collocation points

# Generate random collocation points within the domain
key = jax.random.PRNGKey(0)  # Random seed
#x_colloc = jnp.linspace(xmin, xmax, n_colloc).reshape(-1, 1)
x_colloc = jax.random.uniform(key, shape=(n_colloc, 1), minval=xmin, maxval=xmax)

# # Plot the random collocation points
# plt.figure(figsize = (5, 4), dpi = 150)
# #plt.scatter(x_colloc, jnp.zeros_like(x_colloc), color='black', label='Collocation Points', s=5)
# plt.xlabel('$x$')
# plt.legend()
# #plt.ylabel('Random Points (Plotted on $y=0$)')
# plt.title('Random Collocation Points in 1D Domain')


## 3. Define PDE Residual

In [14]:
# PDE Residual for 1D Darcy's Flow
def pde_residual_1d_darcy(x, h):
    """
    Compute the PDE residual for Darcy's flow.

    Args:
        h Lambda function representing the neural network solution h(x).
        x: Collocation points (array of shape Nx1).
    
    Returns:
        pde_res: PDE residual (array of shape Nx1).
    """

    h_x = lambda x: vmap(jacfwd(h))(x)  # First derivative
    
    h_xx = lambda x: vmap(jacrev(h_x))(x)  # Second derivative
    
    pde_res = h_xx(x).reshape(-1, 1)
    
    return pde_res

## 4. Define Neural Network Architecture

In [15]:
# Function to initialize parameters of the neural network (weights and biases)
def init_mlp(layers, key):
    """
    Initialize the neural network using Xavier initialization.

    Args:
        layers: List defining the number of nodes in each layer (e.g., [1, 50, 50, 1]).
        key: Random key for initialization.
    
    Returns:
        params: Initialized weights and biases for the network.
    """
    params = []
    for i in range(len(layers) - 1):
        key, subkey = random.split(key)
        bound = jnp.sqrt(6. / (layers[i] + layers[i+1]))
        weights = random.uniform(subkey, (layers[i], layers[i+1]), minval=-bound, maxval=bound)
        biases = jnp.zeros(layers[i+1])
        params.append((weights, biases))
    return params



# Define Neural Network Architecture 
def neural_net(params, coord):
    """
    Forward pass through the neural network.

    Args:
        params: Neural network parameters (list of weights and biases).
        coord: Input coordinates (array of shape Nx1).
    
    Returns:
        h_nn: Output of the neural network.
    """
    for w, b in params:
        h_nn = jnp.dot(coord, w) + b
        coord = jnp.tanh(h_nn)  # Activation function
    return h_nn

## 5. Define Loss Function

In [16]:
# Loss Function
def loss_func(params, x, bcs):
    """
    Compute the total loss for the PINN.

    Args:
        params: Neural network parameters.
        x: Collocation points (array of shape Nx1).
        bcs: Boundary conditions [left_head, right_head].
    
    Returns:
        loss: Total loss (scalar).
    """

    h = lambda x: neural_net(params, x)
    
    # PDE residual loss
    #pde_res = pde_residual_1d_darcy(params, x)
    pde_res = pde_residual_1d_darcy(x, h)
    
    pde_loss = jnp.mean(pde_res**2)  # Ensure this is scalar

    # Boundary condition loss
    left_bc_loss = (h(xmin) - bcs[0])**2
    
    right_bc_loss = (h(xmax) - bcs[1])**2
    
    bc_loss = left_bc_loss + right_bc_loss  # Ensure scalar addition

    # Total loss
    loss = pde_loss + bc_loss
    
    return jnp.sum(loss)  # Return scalar

## 6. Define Hyperparameters and Training Loop
This cell contains the training loop for the Physics-Informed Neural Network (PINN). It includes functions and code to update the network parameters, compute the loss, track training progress, and identify the best-performing model during the training process.

In [17]:
# Training PINN
n_hl = 3  # Number of hidden layers
n_nodes = 50  # Number of nodes per layer
lr = 1E-2  # Learning rate
epoch = 5000  # Number of epochs

layers = [1, n_nodes, 1]  # Neural network architecture
soln_type = 1  # Solution type: 0 = homogeneous, 1 = heterogeneous

# Initialize neural network
key = random.PRNGKey(1234)
params = init_mlp(layers, key)

# Initialize optimizer
optimizer = optax.adam(learning_rate=lr)
opt_state = optimizer.init(params)


# Parameters update function
@jit
def update(params, opt_state, x):
    """
    Perform one optimization step.

    Args:
        params: Neural network parameters.
        opt_state: Optimizer state.
        x: Collocation points.
    
    Returns:
        value: Current loss.
        params: Updated parameters.
        opt_state: Updated optimizer state.
    """
    value, grads = jax.value_and_grad(loss_func)(params, x, bcs)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return value, params, opt_state


#%% Training loop
# Initialize lists to track all losses and epochs
all_losses = []
all_epochs = []

# Initialize tracking variables
min_loss = jnp.inf
best_params = None

# for i in range(epoch + 1):
#     loss, params, opt_state = update(params, opt_state, x_colloc)
#     all_losses.append(loss)
#     all_epochs.append(i)
#     if loss < min_loss:
#         min_loss = loss
#         best_params = params
#         best_iteration = i
#     if i % 100 == 0:
#         print(f"Epoch {i}: Loss = {loss:.4e}")

# print(f" ")
# print(f"Best Model:")
# print(f"Best loss: {min_loss:.4e} at iteration {best_iteration}")
# print(f" ")

## 7. Plot pinn loss during training

In [18]:
def plot_pinn_training(all_losses, all_epochs):
    plt.figure(figsize = (5, 4), dpi = 150)
    plt.semilogy(all_epochs, all_losses, '-r', markersize = 4, linewidth = 1.0)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    #plt.legend(prop={"size": fontsize_legend}, loc="best")
    plt.xlim(min(all_epochs), max(all_epochs))
    plt.tight_layout()
    
#plot_pinn_training(all_losses, all_epochs)

## 8. PINN prediction and plotting results

In [19]:
nx = 20
x = np.linspace(xmin, xmax, nx)
x = np.linspace(xmin, xmax, nx).reshape(-1, 1)  # Reshape to (nx, 1) to match neural_net input


# Predict head using the trained neural network
# h_pinn = neural_net(best_params, x)  # Output will be (nx, 1)
# print(h_pinn)

In [20]:
def plot_head_solutions(coords, pinn_head):
    plt.figure(figsize = (5, 4), dpi = 150)
    #plt.plot(coords, head_exact, color='Black', linestyle='-', linewidth = 1.25, label="Analytical")
    plt.plot(coords, pinn_head, '--', color='red', markersize = 4,  markerfacecolor = 'green', linewidth = 1.25, label="PINN")
    plt.xlabel("X [L]")
    plt.ylabel("Head [L]")
    plt.title("PINN solution for head")
    plt.xlim(np.min(coords), np.max(coords))
    plt.tight_layout()
#plot_head_solutions(x, h_pinn )

## 9. Solution verifications

In [21]:
# analytical solution of this homogeneous 1D case
def exact_solution_homo(x_coords):
    head_exact = -0.1*x_coords + 1
    return head_exact  

# h_exact = exact_solution_homo(x)
# print(h_exact)

In [22]:
def plot_head_solutions_as_pinn(coords, pinn_head, exact_head):
    plt.figure(figsize = (5, 4), dpi = 150)
    plt.plot(coords, exact_head, color='Black', linestyle='-', linewidth = 1.0, label="Analytical")
    plt.plot(coords, pinn_head, '--', color='red', markersize = 4,  markerfacecolor = 'green', linewidth = 1.5, label="PINN")
    plt.xlabel("X [L]")
    plt.ylabel("Head [L]")
    plt.legend(loc="upper right")
    plt.xlim(np.min(coords), np.max(coords))
    plt.tight_layout()
#plot_head_solutions_as_pinn(x, h_pinn, h_exact)

## 10. Error analysis between the analytical solution and PINN prediction

In [13]:
# Analytical solution and PINN predictions are already 1D
analytical_Pr = h_exact.flatten()  # Flatten the analytical solution (if necessary)
pinn_Pr = h_pinn.flatten()  # Flatten the PINN predictions (if necessary)

# Perform regression
reg = LinearRegression()
reg.fit(analytical_Pr.reshape(-1, 1), pinn_Pr)  # Fit the model

# Predict using the regression model
y_pred = reg.predict(analytical_Pr.reshape(-1, 1))

# Calculate R^2 score
r2 = r2_score(pinn_Pr, y_pred)

# Calculate RMSE
mse = mean_squared_error(pinn_Pr, y_pred)  # Mean Squared Error
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Print results
print(f"R^2 Score: {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

R^2 Score: 1.0000
Root Mean Squared Error (RMSE): 0.0000


In [23]:
# #%% Plotting Regression
# plt.figure(figsize = (5, 4), dpi = 150)
# plt.scatter(analytical_Pr, pinn_Pr, label="Data Points", color="blue", alpha=0.7)
# plt.plot(analytical_Pr, y_pred, label="Regression Line", color="red", linewidth=2)

# # Add annotations for RMSE and R^2
# plt.text(
#     0.05,
#     0.95,
#     f"$R^2$ = {r2:.4f}\nRMSE = {rmse:.4f}",
#     transform=plt.gca().transAxes,
#     fontsize=12,
#     verticalalignment="top",
#     bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
# )

# plt.xlabel("Analytical Solution")
# plt.ylabel("PINN Prediction")
# plt.title("Regression Plot: Analytical vs. PINN Predictions")
# plt.legend(loc="lower right")
# plt.show()

In [24]:
# #%% Plotting Error
# # Calculate percentage error
# error_percentage = 100 * (analytical_Pr - pinn_Pr) / analytical_Pr
# plt.figure(figsize = (5, 4), dpi = 150)
# plt.plot(x, error_percentage, label="Error (%)", color="red", marker="o")
# plt.xlabel("$x$")
# plt.ylabel("$Error (\%)$")
# plt.title("Percentage Error Between Analytical and PINN Solution")
# plt.legend()
# plt.show()